## TD 1- Reinforcement Learning - Magali Morin

### Question 1 

All modules are installed on my env.

In [1]:
import gym.envs.toy_text.frozen_lake as fl
import numpy as np
import matplotlib.pyplot as plt

### Question 2 

Description of the MDP : 

States : [0;15]

Action : 0 LEFT , 1 DOWN , 2 RIGHT, 3 UP

Rewards : 0 for states 0 to 14 and 1 for state 15

Transitions P : When choosing the action x ∈ [0,3] there is a probability 1/3 to move in the direction x, a probability 1/3 to move in the direction x+1[4] and a probability 1/3 to move in the direction x-1[4]. Furthermore, you can't move towards the boundaries of the lake.

### Question 3

Choose right policy

### Question 4

Monte Carlo estimation of the value fonciton

In the Monte Carlo estimation, we update the state function at the end of each episode.

In [5]:
env = fl.FrozenLakeEnv()
env.render() 


SFFF
FHFH
FFFH
HFFG


In [17]:
NBR_EPISODES = 100000
HORIZON = 200
GAMMA = 0.9
SEED = env.seed(42)

def expected_value_method(policy): 
    """
    policy : array of len nb of states which gives the action to do for each state
    return : an value vector with the value to do an action for each state
    """
    value_fnct = np.zeros(env.nS)
    for state in range(env.nS): 
        state_value= 0
        env.isd = np.zeros(env.nS)
        env.isd[state] = 1
        env.reset()
        for i in range(NBR_EPISODES) : 
            time = 0
            env.reset()
            done = False
            discount = 1
            while (not done) and (time<HORIZON) : 
                time += 1
                current_state, reward, done, info = env.step(policy[state])
                state_value += discount * reward
                discount = discount * GAMMA
        value_fnct[state] = state_value / NBR_EPISODES
    return(value_fnct)    

### Question 5 : Linear system method

The equation for the linear system method is : $$V^{\pi} = r_{\pi}(I-\gamma P)^{-1}$$

with P : the dynamic transition probabilities matrix. P[a][b] is the probability to pass from state a to state b.

     I : Identity vector 
     
     R : reward vecor R[a] is the reward given for being is state a.

In [39]:
right_policy = np.array([fl.RIGHT for _ in range(env.nS)])

def linear_system_method(policy):
    """
    policy : array of len nb of states which gives the action to do for each state
    return : an value vector with the value to do an action for each state
    """
    #Create the matrix 
    I = np.identity(env.nS)
    P = np.zeros((env.nS, env.nS))
    R = np.zeros(env.nS)
    
    for state in range(env.nS):
        transitions = env.P[state][policy[state]]
        for transition in transitions : 
            proba = transition[0]
            next_state = transition[1]
            reward = transition[2]
            P[state, next_state] += proba
            R[state] += reward*proba
    value_fnct = np.dot(np.linalg.inv(I-GAMMA*P),R)
    return(value_fnct)

We can compare the result of the two policies by calculating the value function in each case for the right policy.

In [42]:
right_policy = np.array([fl.RIGHT for i in range(env.nS)])

linear  = linear_system_method(right_policy)
monte_carlo = expected_value_method(right_policy)
print(linear)
print(monte_carlo)

delta  = (linear-monte_carlo).mean()

print(delta)

[1.30776757e-02 1.17595819e-02 2.74390244e-02 1.15266041e-16
 1.87549947e-02 1.09819425e-16 6.40243902e-02 1.53688055e-16
 4.94389734e-02 1.46041583e-01 1.85975610e-01 0.00000000e+00
 0.00000000e+00 3.00829668e-01 5.55894309e-01 0.00000000e+00]
[0.01312729 0.01141018 0.026981   0.         0.01894528 0.
 0.06315667 0.         0.04905123 0.14661202 0.18430732 0.
 0.         0.30034573 0.55512608 0.        ]
0.0002608123110508956


We observe that the difference is very small. The big difference is the time of calculation of both algorithms. Linear method is much faster than Monte Carlo.

### Question 6 : Bellman operator method

In [43]:
def bellman_operator(policy):
    """
    policy : array of len nb of states which gives the action to do for each state
    return : the bellman operator to apply to a value function
    """
    P = np.zeros((env.nS, env.nS))
    R = np.zeros(env.nS)
    
    for state in range(env.nS):
        transitions = env.P[state][policy[state]]
        for transition in transitions : 
            proba = transition[0]
            next_state = transition[1]
            reward = transition[2]
            P[state, next_state] += proba
            R[state] += reward*proba
    return(lambda value : R + GAMMA*np.dot(P,value))

In [53]:
def bellman_method(policy, epsilon):
    """
    policy : array of len nb of states which gives the action to do for each state
    epsilon : stopping criteria, normative difference of two consecutive value function
    return : the value function for a given policy
    """
    value = np.zeros(env.nS)
    next_value = bellman_operator(policy)(value)
    
    while np.linalg.norm(value - next_value) > epsilon : 
        value = next_value
        next_value = bellman_operator(policy)(value)
    
    return(next_value)

In [54]:
epsilon = 1e-5
print(bellman_method(right_policy, epsilon))

[0.01306594 0.01175684 0.02743628 0.         0.01874559 0.
 0.06402165 0.         0.04943158 0.14603605 0.18597287 0.
 0.         0.30082135 0.55589157 0.        ]


The stopping criterio used was the threshold. We could also add an maximal number of iteration to be sure the loop stops.

### Question 7 : optimal bellman operator


Mathematical definition:

$$ \mathcal{T}[v] = \max_{a} [R_a + \gamma P_av]$$ 

In [66]:
def bellman_optimal_operator(value, R, P):
    """
    value : value function
    R : reward vector
    P : dynamic transition probabilities
    Returns : optimal bellman operator
    """
    next_value = np.zeros(env.nS)
    for state in range(env.nS) : 
        best_s = 0
        for a in range(env.nA) : 
            value_a = R[state][a] + GAMMA * np.dot(P[:,:,a],value)[state]
            if value_a > best_s : 
                best_s = value_a
        next_value[state] = best_s
    return(next_value)
    

In [67]:
def bellman_optimal_method(epsilon):
    """
    epsilon : stopping criteria, normative difference of two consecutive value function
    return : an approximation of the value function
    """
    
    #We calculate the transition matrix and the reward vector 
    R = np.zeros((env.nS, env.nA))
    P = np.zeros((env.nS,env.nS,env.nA))

    for state in range(env.nS):
        for action in range(env.nA):
            transitions = env.P[state][action]
            for transition in transitions : 
                proba = transition[0]
                next_state = transition[1]
                reward = transition[2]
                P[state, next_state, action] += proba
                R[state, action] += reward*proba
    
    value = np.zeros(env.nS)
    next_value = bellman_optimal_operator(value,R,P)
    while np.linalg.norm(value - next_value) > epsilon : 
        value = next_value
        next_value = bellman_optimal_operator(value,R,P)
    
    return(next_value)                     

In [68]:
epsilon = 1e-5
print(bellman_optimal_method(epsilon))

[0.06885897 0.06138751 0.07439003 0.05578562 0.0918255  0.
 0.11219759 0.         0.14541285 0.24748111 0.29960644 0.
 0.         0.37992446 0.63901416 0.        ]


### Question 8 : Value iteration method

Mathematical definition

$$\mathcal{G}[v] = argmax_{a} [R_a + \gamma P_av]$$

In [69]:
def bellman_optimal_policy(value,R,P):
    """
    value : value function
    R : reward vector
    P : dynamic transition probabilities
    returns : Bellman policy
    """
    policy = np.zeros(env.nS)
    for state in range(env.nS) : 
        best_s = 0
        best_a = 0
        for a in range(env.nA) : 
            value_a = R[state][a] + GAMMA * np.dot(P[:,:,a],value)[state]
            if value_a > best_s : 
                best_s = value_a
                best_a = a
        policy[state] = best_a
    return(policy)

In [70]:
def value_iteration_method(epsilon): 
    """
    epsilon : stopping criteria, normative difference of two consecutive value function
    return : an approximation of the optimal policy
    """
    
    #We calculate the transition matrix and the reward vector 
    R = np.zeros((env.nS, env.nA))
    P = np.zeros((env.nS,env.nS,env.nA))

    for state in range(env.nS):
        for action in range(env.nA):
            transitions = env.P[state][action]
            for transition in transitions : 
                proba = transition[0]
                next_state = transition[1]
                reward = transition[2]
                P[state, next_state, action] += proba
                R[state, action] += reward*proba
    
    value = bellman_optimal_method(epsilon)
    policy = bellman_optimal_policy(value,R,P)
    
    return(policy)

In [71]:
value_iteration_method(epsilon)

array([0., 3., 0., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

### Question 9 : Policy iteration method

Mathematical demonstration : 

$\pi_0$ initialized arbitrarily.

$v_{n+1} = (I-\gamma P_{\pi_n})^{-1}R_{\pi_n}$.

$\pi_{n+1} = \mathcal{G}[v_n]$.

Stoping criteria: $|v_{n+1} - v_{n}| < \epsilon$.

In [90]:
def policy_iteration_method(epsilon):
    '''
    epsilon : stopping criteria, normative difference of two consecutive value function
    Return : optimal policy

    '''
    policy = np.random.randint(0,env.nA,env.nS)
    value = linear_system_method(policy) #or bellman method
    
    #We calculate R and P for this environment
    R = np.zeros((env.nS, env.nA))
    P = np.zeros((env.nS,env.nS,env.nA))

    for state in range(env.nS):
        for action in range(env.nA):
            transitions = env.P[state][action]
            for transition in transitions : 
                proba = transition[0]
                next_state = transition[1]
                reward = transition[2]
                P[state, next_state, action] += proba
                R[state, action] += reward*proba
    
    next_policy = bellman_optimal_policy(value,R,P)
    next_value = linear_system_method(policy)
    while np.linalg.norm(value - next_value) > epsilon :
        policy = next_policy
        value = next_value
        next_policy = bellman_optimal_policy(value,R,P)
        next_value = linear_system_method(next_policy)
    
    return(next_policy) 

In [91]:
epsilon = 1e-5
policy_iteration_method(epsilon)

array([0., 3., 0., 3., 0., 0., 2., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

### Question 10 : Comparison of both methods

The two methods are giving similar results.
Tu push the comparison further, we could compute the time to see which method is the quickest.

### Question 11 : 
Using the contraction property of the optimal Bellman operator, implement a function
that iteratively applies the optimal Bellman operator to compute the optimal value function
Q*.

In [103]:
def q_operator(value,R,P):
    next_q = np.zeros((env.nS,env.nA))
    for state in range(env.nS):
        for action in range(env.nA):
            reward = R[state,action]
            for state_bis in range(env.nS):
                optim_value = 0
                for action_bis in range(env.nA):
                    value_a = value[state_bis,action_bis]
                    if value_a > optim_value:
                        optim_value = value_a
                reward += GAMMA * P[state,state_bis,action] * optim_value
            next_q[state,action] = reward
    return(next_q)   

In [110]:
def q_method(epsilon):
    """
    epsilon:stopping criteria
    Returns optimal quality function
    """
    #We calculate R and P for this environment
    R = np.zeros((env.nS, env.nA))
    P = np.zeros((env.nS,env.nS,env.nA))

    for state in range(env.nS):
        for action in range(env.nA):
            transitions = env.P[state][action]
            for transition in transitions : 
                proba = transition[0]
                next_state = transition[1]
                reward = transition[2]
                P[state, next_state, action] += proba
                R[state, action] += reward*proba
                
    q = np.zeros((env.nS,env.nA))
    next_q = q_operator(q, R, P)
    while np.linalg.norm(q-next_q) > epsilon:
        q = next_q
        next_q = q_operator(q,R,P)
        
    return(next_q)   

In [111]:
q_method(epsilon)

array([[0.06887237, 0.06663045, 0.06663045, 0.05974078],
       [0.03907988, 0.04297989, 0.04073797, 0.06139887],
       [0.07439831, 0.06881719, 0.0727172 , 0.05747583],
       [0.03905686, 0.03905686, 0.03347574, 0.05579473],
       [0.09183769, 0.07117679, 0.06428712, 0.04821147],
       [0.        , 0.        , 0.        , 0.        ],
       [0.11220205, 0.08988305, 0.11220205, 0.02231899],
       [0.        , 0.        , 0.        , 0.        ],
       [0.07117679, 0.11787214, 0.1017965 , 0.14542271],
       [0.15760471, 0.24748776, 0.20386154, 0.13350927],
       [0.29961112, 0.26595078, 0.22536519, 0.10790627],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.18822442, 0.30568334, 0.37992926, 0.26595078],
       [0.39556639, 0.63901667, 0.61492124, 0.53719488],
       [0.        , 0.        , 0.        , 0.        ]])

### Question 12 : Optimal policy for Q

$$\pi* = argmax_{a}[q(s,a)]$$

In [114]:
def get_policy_q(epsilon):
    q = q_method(epsilon)
    p = np.zeros(env.nS)
    for state in range(env.nS):
        best_q = 0
        best_a = 0
        for action in range(env.nA):
            if q[state,action] > best_q:
                best_q = q[state,action]
                best_a = action
        p[state] = best_a
    return(p)

In [115]:
get_policy_q(epsilon)

array([0., 3., 0., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

The results are similar to question 10.

### Question 13 : Trajectory

In [144]:

def final_trajectory(policy):
    """
    policy : a given policy
    Return : trajectory
    """
   
    env.reset()
    done = False
    t = 0
    rewards=0
    discount = 1
    while (not done) and (t < HORIZON):
        next_state, r, done, _ = env.step(fl.RIGHT)
        rewards += discount * r
        discount *= GAMMA
        t += 1
    if done and env.nS != env.nS-1: 
        return('Too bad, you lost')
    elif done : 
        return('You won and your reward is ', rewards)
        

In [145]:
final_trajectory(policy_iteration_method(epsilon))

'Too bad, you lost'